# Статистики за корона вирусот во Македонија

In [14]:
import pandas as pd
# import nbinteract as nbi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, fixed

%matplotlib inline

In [2]:
all_data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")

all_data.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,50,50,53,59,70,75,82,114,147,177
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,502,511,581,639,639,701,773,839,825,878
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,150,150,160,178,178,200,212,226,243,266
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,99,117,129,149,149,197,238,428,566,673


In [3]:
def get_data_by_country_name(data, country_column, country_name):
    return data.loc[data[country_column] == country_name]


def convert_data_in_day_count_larger_than_zero_format(data, id_vars, value_vars, var_name, value_name):
    converted = pd.melt(data, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)
    converted = converted.loc[converted[value_name] > 0].reset_index(drop=True)
    converted.insert(0, 'artificial_day', range(1, 1 + len(converted)))
    return converted.loc[converted[value_name] > 0].reset_index(drop=True)

def get_data_for_country(data, name):
    res = get_data_by_country_name(data=data, country_column='Country/Region', country_name=name)
    res = convert_data_in_day_count_larger_than_zero_format(data=res, id_vars=res.columns[1 : 4], value_vars=res.columns[4:], var_name='date', value_name='count')
    return res


In [17]:
def get_data_for_countries(data, countries):
    data = [get_data_for_country(data, country) for country in countries]
    lengths = [len(country_data) for country_data in data]
    comparable_days_count = max(lengths)
#     interesting_data = pd.concat([country_data.iloc[:comparable_days_count] for country_data in data])
    return data, comparable_days_count

def draw_line_plot(day_range, data):
    plt.close('all')
    figure = plt.figure(num='figure', figsize=(15,10))
    from_day = day_range[0]
    to_day = day_range[1]
    print(from_day)
    interesting_data = pd.concat([country_data.loc[(country_data['artificial_day'] >= from_day) & 
                                                   (country_data['artificial_day'] <= to_day)] 
                                  for country_data in data])
    line_plot = sns.lineplot(x="artificial_day", y="count", hue='Country/Region', data=interesting_data, ax=plt.axes())

    line_plot.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    line_plot.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    
    return line_plot


In [18]:
interesting_data, max_days = get_data_for_countries(all_data, countries=['North Macedonia', 'Spain', 'Croatia', 'Romania'])

data = interesting_data

day_range_chooser = widgets.IntRangeSlider(
    value=[1, 10],
    min=1,
    max=max_days,
    step=1,
    description='Избери за кои денови:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# day_range_chooser = widgets.IntSlider(
#     value=1,
#     min=1,
#     max=max_days - 10,
#     step=1,
#     description='Избери за кои денови:',
#     disabled=False,
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
# )

# display(day_range_chooser)

# def on_values_change(change):
#     draw_line_plot(interesting_data, change['new'])


# draw_line_plot(interesting_data, (1, 10))
# day_range_chooser.observe(on_values_change, names='value')



interact(draw_line_plot, day_range=day_range_chooser, data=fixed(interesting_data))
# line_plot.set_xticklabels(line_plot.get_xticklabels(), rotation=45)
# line_plot.xaxis.set_major_locator(ticker.MultipleLocator(2))

IntRangeSlider(value=(1, 10), continuous_update=False, description='Избери за кои денови:', max=46, min=1)

interactive(children=(IntRangeSlider(value=(1, 10), continuous_update=False, description='Избери за кои денови…

<function __main__.draw_line_plot(day_range, data)>